## Machine Learning Translation

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM, Dense
import numpy as np

In [2]:
batch_size = 64 ## Batch Size for training
epochs = 100 ## number of epoch to train
latent_dim = 25 ## Latent Dimensionality of the encoding space
num_samples = 10000 ## number of sample to train

## vectorize the data

In [3]:
input_texts = [] ## English Sentence
target_texts = []  ## French Sentence
input_characters = set() ## unique input char
target_characters = set() ## unique targe char
with open('fra.txt','r',encoding='utf-8') as f:
        lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines) - 1)]:
    input_text,target_text, _ = line.split('\t') 
    ## We use "Tab" as the "Start Sequence" characters
    ## for the target, and "\n" as "end sequence" characters
    target_text = '\t' +target_text+ '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)   

In [5]:
len(input_characters)

71

In [6]:
len(target_characters)

92

In [7]:
print(input_texts, end =' ') 

['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.', 'Stop!', 'Stop!', 'Stop!', 'Wait!', 'Wait!', 'Go on.', 'Go on.', 'Go on.', 'Hello!', 'Hello!', 'I see.', 'I try.', 'I won!', 'I won!', 'I won.', 'Oh no!', 'Attack!', 'Attack!', 'Cheers!', 'Cheers!', 'Cheers!', 'Cheers!', 'Get up.', 'Go now.', 'Go now.', 'Go now.', 'Got it!', 'Got it!', 'Got it?', 'Got it?', 'Got it?', 'Hop in.', 'Hop in.', 'Hug me.', 'Hug me.', 'I fell.', 'I fell.', 'I know.', 'I left.', 'I left.', 'I lied.', 'I lost.', 'I paid.', "I'm 19.", "I'm OK.", "I'm OK.", 'Listen.', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'Really?', 'Really?', 'Really?', 'Thanks.', 'We try.', 'We won.', 'We won.', 'We won.', 'We won.', 'Ask Tom.', 'Awesome!', 'Be calm.', 'Be calm.', 'Be calm.', 'Be cool.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be kind.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.

In [8]:
print(target_texts, end =' ') 

['\tVa !\n', '\tSalut !\n', '\tSalut.\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tQui ?\n', '\tÇa alors\u202f!\n', '\tAu feu !\n', "\tÀ l'aide\u202f!\n", '\tSaute.\n', '\tÇa suffit\u202f!\n', '\tStop\u202f!\n', '\tArrête-toi !\n', '\tAttends !\n', '\tAttendez !\n', '\tPoursuis.\n', '\tContinuez.\n', '\tPoursuivez.\n', '\tBonjour !\n', '\tSalut !\n', '\tJe comprends.\n', "\tJ'essaye.\n", "\tJ'ai gagné !\n", "\tJe l'ai emporté !\n", '\tJ’ai gagné.\n', '\tOh non !\n', '\tAttaque !\n', '\tAttaquez !\n', '\tSanté !\n', '\tÀ votre santé !\n', '\tMerci !\n', '\tTchin-tchin !\n', '\tLève-toi.\n', '\tVa, maintenant.\n', '\tAllez-y maintenant.\n', '\tVas-y maintenant.\n', "\tJ'ai pigé !\n", '\tCompris !\n', '\tPigé\u202f?\n', '\tCompris\u202f?\n', "\tT'as capté\u202f?\n", '\tMonte.\n', '\tMontez.\n', '\tSerre-moi dans tes bras !\n', '\tSerrez-moi dans vos bras !\n', '\tJe suis tombée.\n', '\tJe suis tombé.\n', '\tJe sais.\n', '\tJe suis parti.\n', '\tJe suis partie.\n', "\tJ'ai menti.\n", "\

In [15]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_token=len(input_characters)
num_decoder_token=len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [16]:
print("Number of samples",len(input_texts))
print("Number of unique input token",num_encoder_token)
print("Number of unique output token",num_decoder_token)
print("max seq length for input",max_encoder_seq_length)
print("max seq length for output",max_decoder_seq_length)

Number of samples 10000
Number of unique input token 71
Number of unique output token 92
max seq length for input 16
max seq length for output 59


## Add index for input and target characters

In [19]:
input_token_index = dict([char,i] for i,char in enumerate(input_characters))
target_token_index = dict([char,i] for i,char in enumerate(target_characters))

In [35]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_token),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_token),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_token),
    dtype='float32')

## Above logic Example

In [31]:
np.zeros((5,4,1),dtype='float32')

array([[[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)

## one hot encoding

In [41]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t,target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1,target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [43]:
encoder_input_data[0].shape

(16, 71)

In [45]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_token))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [47]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_token))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_token, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [48]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 11s 87ms/step - loss: 2.3186 - val_loss: 1.3914
Epoch 2/100
125/125 [==============================] - 10s 79ms/step - loss: 1.1219 - val_loss: 1.1438
Epoch 3/100
125/125 [==============================] - 9s 74ms/step - loss: 0.9813 - val_loss: 1.0597
Epoch 4/100
125/125 [==============================] - 10s 79ms/step - loss: 0.9097 - val_loss: 0.9881
Epoch 5/100
125/125 [==============================] - 9s 71ms/step - loss: 0.8523 - val_loss: 0.9274
Epoch 6/100
125/125 [==============================] - 9s 72ms/step - loss: 0.7969 - val_loss: 0.8697
Epoch 7/100
125/125 [==============================] - 9s 74ms/step - loss: 0.7460 - val_loss: 0.8182
Epoch 8/100
125/125 [==============================] - 8s 62ms/step - loss: 0.7029 - val_loss: 0.7784
Epoch 9/100
125/125 [==============================] - 9s 74ms/step - loss: 0.6690 - val_loss: 0.7470
Epoch 10/100
125/125 [==============================] - 9s 72ms/step - loss: 0.

125/125 [==============================] - 10s 83ms/step - loss: 0.4178 - val_loss: 0.5137
Epoch 81/100
125/125 [==============================] - 10s 84ms/step - loss: 0.4170 - val_loss: 0.5119
Epoch 82/100
125/125 [==============================] - 9s 74ms/step - loss: 0.4162 - val_loss: 0.5118
Epoch 83/100
125/125 [==============================] - 9s 74ms/step - loss: 0.4155 - val_loss: 0.5118
Epoch 84/100
125/125 [==============================] - 9s 74ms/step - loss: 0.4147 - val_loss: 0.5126
Epoch 85/100
125/125 [==============================] - 10s 76ms/step - loss: 0.4140 - val_loss: 0.5103
Epoch 86/100
125/125 [==============================] - 10s 82ms/step - loss: 0.4134 - val_loss: 0.5100
Epoch 87/100
125/125 [==============================] - 10s 77ms/step - loss: 0.4126 - val_loss: 0.5092
Epoch 88/100
125/125 [==============================] - 9s 74ms/step - loss: 0.4119 - val_loss: 0.5084
Epoch 89/100
125/125 [==============================] - 10s 77ms/step - loss: 0.4

In [57]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [63]:
reverse_input_char_index = dict(
 (i,char) for char,i in input_token_index.items()
)
reverse_target_char_index = dict(
 (i,char) for char,i in target_token_index.items()
)

In [73]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_token))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_token))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [78]:
for seq_index in range(10):
    ## take sentence part of trainng set 
    ## for trying out decoding
    input_seq=encoder_input_data[seq_index:seq_index+1]
    decode_sentence = decode_sequence(input_seq)
    print("===========")
    print("Input Sentence",input_texts[seq_index])
    print("Decoded Sentence",decode_sentence)

Input Sentence Go.
Decoded Sentence Pars !

Input Sentence Hi.
Decoded Sentence Artez pas !

Input Sentence Hi.
Decoded Sentence Artez pas !

Input Sentence Run!
Decoded Sentence oonnez-le !

Input Sentence Run!
Decoded Sentence oonnez-le !

Input Sentence Who?
Decoded Sentence Qui vous ?

Input Sentence Wow!
Decoded Sentence Comme pas !

Input Sentence Fire!
Decoded Sentence Ounis !

Input Sentence Help!
Decoded Sentence Priez le main !

Input Sentence Jump.
Decoded Sentence Sourez !

